<a href="https://colab.research.google.com/github/newmantic/KMV/blob/main/KMV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve

def estimate_asset_value_and_volatility(E, sigma_E, D, r, T):
    """
    Estimate the asset value (V) and asset volatility (sigma_V) from the equity value (E),
    equity volatility (sigma_E), debt (D), risk-free rate (r), and time horizon (T).

    :param E: Market value of equity
    :param sigma_E: Volatility of equity
    :param D: Value of debt (liabilities)
    :param r: Risk-free rate
    :param T: Time horizon (usually 1 year)
    :return: Estimated asset value (V) and asset volatility (sigma_V)
    """
    def equations(p):
        V, sigma_V = p
        d1 = (np.log(V / D) + (r + 0.5 * sigma_V ** 2) * T) / (sigma_V * np.sqrt(T))
        d2 = d1 - sigma_V * np.sqrt(T)
        return [
            V * norm.cdf(d1) - D * np.exp(-r * T) * norm.cdf(d2) - E,
            V * norm.cdf(d1) * sigma_V - sigma_E * E
        ]

    V, sigma_V = fsolve(equations, (E + D, sigma_E))
    return V, sigma_V

def calculate_distance_to_default(V, sigma_V, D, r, T):
    """
    Calculate the Distance to Default (DD) for a firm.

    :param V: Estimated asset value
    :param sigma_V: Estimated asset volatility
    :param D: Value of debt (liabilities)
    :param r: Risk-free rate
    :param T: Time horizon (usually 1 year)
    :return: Distance to Default (DD)
    """
    d1 = (np.log(V / D) + (r + 0.5 * sigma_V ** 2) * T) / (sigma_V * np.sqrt(T))
    d2 = d1 - sigma_V * np.sqrt(T)
    return d2

def calculate_edf(DD):
    """
    Calculate the Expected Default Frequency (EDF) using the Distance to Default (DD).

    :param DD: Distance to Default
    :return: Expected Default Frequency (EDF)
    """
    return norm.cdf(-DD)

# Example

# Parameters
E = 100        # Market value of equity
sigma_E = 0.25  # Volatility of equity
D = 80         # Value of debt (liabilities)
r = 0.03       # Risk-free rate
T = 1          # Time horizon (1 year)

# Estimate asset value and volatility
V, sigma_V = estimate_asset_value_and_volatility(E, sigma_E, D, r, T)
print(f"Estimated Asset Value: V = {V:.2f}, Estimated Asset Volatility: sigma_V = {sigma_V:.2f}")

# Calculate Distance to Default
DD = calculate_distance_to_default(V, sigma_V, D, r, T)
print(f"Distance to Default: DD = {DD:.2f}")

# Calculate Expected Default Frequency
EDF = calculate_edf(DD)
print(f"Expected Default Frequency: EDF = {EDF:.4f}")

Estimated Asset Value: V = 177.64, Estimated Asset Volatility: sigma_V = 0.14
Distance to Default: DD = 5.81
Expected Default Frequency: EDF = 0.0000
